<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">

# Futures - non-blocking distributed calculations

Submit arbitrary functions for computation in a parallelized, eager, and non-blocking way. 

The `futures` interface (derived from the built-in `concurrent.futures`) provide fine-grained real-time execution for custom situations. We can submit individual functions for evaluation with one set of inputs, or evaluated over a sequence of inputs with `submit()` and `map()`. The call returns immediately, giving one or more *futures*, whose status begins as "pending" and later becomes "finished". There is no blocking of the local Python session.

With futures, as soon as the inputs are available and there is compute available, the computation starts. 

- Delayed = lazy
- Futures = eager

In [1]:
from dask.distributed import Client

client = Client(n_workers=4)
client

/Users/nollde/tools/miniforge3/envs/dask-tutorial/lib/python3.14/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53632 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:53632/status,
Dashboard: http://127.0.0.1:53632/status,Workers: 4
Total threads: 12,Total memory: 24.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53633,Workers: 0
Dashboard: http://127.0.0.1:53632/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:53647,Total threads: 3
Dashboard: http://127.0.0.1:53651/status,Memory: 6.00 GiB
Nanny: tcp://127.0.0.1:53636,


## Basics

Just like we did in the delayed notebook, let's make some toy functions, `inc` and `add`, that sleep for a while to simulate work. We'll then time running these functions normally.

In [2]:
from time import sleep


def inc(x):
    sleep(1)
    return x + 1


def double(x):
    sleep(2)
    return 2 * x


def add(x, y):
    sleep(1)
    return x + y

We can run these locally

In [3]:
inc(1)

2

Or we can submit them to run remotely with Dask. This immediately returns a future that points to the ongoing computation, and eventually to the stored result.

In [4]:
future = client.submit(inc, 1)  # returns immediately with pending future
future

<Future: pending, key: inc-2b76f0c31eae0ad9cb068c6b53d6580b>

If you wait a second, and then check on the future again, you’ll see that it has finished.

In [5]:
future

<Future: finished, type: int, key: inc-2b76f0c31eae0ad9cb068c6b53d6580b>

You can block on the computation and gather the result with the `.result()` method.

In [6]:
future.result()

2

#### Other ways to wait for a future
```python
from dask.distributed import wait, progress
progress(future)
```

shows a progress bar in *this* notebook, rather than having to go to the dashboard. This progress bar is also asynchronous, and doesn't block the execution of other code in the meanwhile.

```python
wait(future)
```
blocks and forces the notebook to wait until the computation pointed to by `future` is done. However, note that if the result of `inc()` is sitting in the cluster, it would take **no time** to execute the computation now, because Dask notices that we are asking for the result of a computation it already knows about. More on this later.

#### Other ways to gather results
```python
client.gather(futures)
```

gathers results from more than one future.

## `client.compute`

Generally, any Dask operation that is executed using `.compute()` or `dask.compute()` can be submitted for asynchronous execution using `client.compute()` instead.

Here is an example from the delayed notebook:

In [7]:
import dask


@dask.delayed
def inc(x):
    sleep(1)
    return x + 1


@dask.delayed
def add(x, y):
    sleep(1)
    return x + y


x = inc(1)
y = inc(2)
z = add(x, y)

So far we have a regular `dask.delayed` output. When we pass `z` to `client.compute` we get a future back and Dask starts evaluating the task graph. 

In [8]:
# notice the difference from z.compute()
# notice that this cell completes immediately
future = client.compute(z)
future

<Future: pending, key: add-487e4294-9393-41ad-b3f8-0ea3323e0f99>

In [9]:
future.result()  # waits until result is ready

5

When using futures, the *computation moves to the data* rather than the other way around, and the client, in the local Python session, need never see the intermediate values.

## `client.submit`

`client.submit` takes a function and arguments, pushes these to the cluster, returning a `Future` representing the result to be computed. The function is passed to a worker process for evaluation. This looks a lot like doing `client.compute()`, above, except now we are passing the function and arguments directly to the cluster.

In [10]:
def inc(x):
    sleep(1)
    return x + 1


future_x = client.submit(inc, 1)
future_y = client.submit(inc, 2)
future_z = client.submit(sum, [future_x, future_y])
future_z

<Future: pending, key: sum-838524d83c5bdcdbf55a7edc90fe0b25>

In [11]:
future_z.result()  # waits until result is ready

5

The arguments to`client.submit` can be regular Python functions and objects, futures from other submit operations or `dask.delayed` objects.

## `client.map`
is very similar to `client.submit` but let's you directly submit a list of tasks, returning a list of futures

In [ ]:
futures = client.map(lambda x: x**2, list(range(1000)))
results = client.gather(futures)

## Why use Futures?

The futures API offers a work submission style that can easily emulate the map/reduce paradigm. If that is familiar to you then futures might be the simplest entrypoint into Dask. 

The other big benefit of futures is that the intermediate results, represented by futures, can be passed to new tasks without having to pull data locally from the cluster. New operations can be setup to work on the output of previous jobs that haven't even begun yet.